In [6]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [7]:
%mkdir ch13
!touch ch13/__init__.py

mkdir: cannot create directory ‘ch13’: File exists


In [0]:
import imp

## 13.0 序论

- 运算符重载的的作用是让用户定义的对象中支持中缀运算符（如 `＋` 和 `｜`） 或一元运算符(如 `-` 和 `~`)

## 13.1 运算符重载基础

- Python 对运算符重载施加了一些限制,做好了灵活性、可用性和安全性方面的平衡
  - 不能重载内置类型的运算符
  - 不能新建运算符,只能重载现有的
  - 某些运算符不能重载—— `is`、`and`、`or` 和 `not`(不过位运算符 `&`、`|` 和 `~` 可以)

## 13.2 一元运算符

- 一元运算符及其对应的特殊方法
  - `-` (`__neg__`)
    - 一元取负运算符
  - `+` (`__pos__`)
    - 一元取正运算符
  - `~` (`__invert__`)
    - 对整数按位取反
  - `abs()` (`__abs__`)
    - 取绝对值
- 一元运算符的基本原则
  - 始终返回一个新对象
  - 即不能修改 self, 要创建并返回合适类型的新类型

###### 示例 13-1 vector_v6.py:把一元运算符 `-` 和 `+` 添加到示例 10-16 中

In [0]:
!cat ch10/vector_v5.py > ch13/vector_v6.py

In [0]:
%%writefile -a ch13/vector_v6.py
  
  
  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)  # 构建一个新的实例


Appending to ch13/vector_v6.py


### `x` 与 `++x` 何时不相等

###### 示例 13-2 算术运算符上下文的变化可能导致 `x` 不等于 `+x`

In [0]:
import decimal
ctx = decimal.getcontext()  # 获取当前全局算术运算的上下文引用
ctx.prec = 40  # 把算术运算上下文的精度设为 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [0]:
one_third == +one_third

True

In [0]:
ctx.prec = 28  # 把精度降低为 28,这是 Python 3.4 为 Decimal 算术运算设定的默认精度
one_third == +one_third

False

In [0]:
+one_third

Decimal('0.3333333333333333333333333333')

###### 示例 13-3 一元运算符 `+` 得到一个新 `Counter` 实例,但是没有零值和负值计数器

- [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter) 中的中缀运算符的作用是把两个 `Counter` 实例的计数器加在一起
- `Counter` 相加时,负值和零值计数会从结果中剔除
- 一元运算符 `+` 等同于加上一个空 `Counter`, 因此它会产生一个新的 `Counter` 且仅保留大于零的计数器

In [0]:
from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [0]:
ct['r'], ct['d'] = -3, 0
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})

In [0]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

## 13.3 重载向量加法运算符

###### 示例 13-4 `Vector.__add__` 方法,第 1 版

In [0]:
from ch13.vector_v6 import Vector

In [0]:
import itertools

def add_v1(self, other):
  pairs = itertools.zip_longest(self, other, fillvalue=0.0)
  return Vector(a + b for a, b in pairs)

Vector.__add__ = add_v1

###### 示例 13-5 第 1 版 `Vector.__add__` 方法也支持 `Vector` 之外的对象

In [0]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [0]:
from ch9.vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

###### 示例 13-6 如果左操作数是 `Vector` 之外的对象,第一版 `Vector.__add__` 方法无法处理

In [0]:
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1

TypeError: ignored

In [0]:
v2d = Vector2d(1, 2)
Vector2d + v1

TypeError: ignored

### Python 中綴特殊运算符的分派机制

- 对于表达式 $a+b$ 来说, 解释器会执行以下几步操作
  1. 如果 `a` 有 `__add__` 方法,而且返回值不是 `NotImplemented`,调用 `a.__add__(b)`,然后返回结果
  2. 如果 `a` 没有 `__add__` 方法,或者调用 `__add__` 方法返回 `NotImplemented`,检查 `b` 有没有 `__radd__` 方法,如果有,而且没有返回 `NotImplemented`,调用 `b.__radd__(a)`,然后返回结果
  3. 如果 `b` 没有 `__radd__` 方法,或者调用 `__radd__` 方法返回 `NotImplemented`,抛出 `TypeError`,并在错误消息中指明操作数类型不支持
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200521164919.png width = 500>


###### 示例 13-7 `Vector.__add__` 和 `__radd__` 方法

In [0]:
def radd_v1(self, other):
  return self + other  # 直接委拖给 __add__

Vector.__radd__ = radd_v1

###### 示例 13-8 `Vector.__add__` 方法的操作数要是可迭代对象

In [0]:
v1 + 1

TypeError: ignored

###### 示例 13-9 `Vector.__add__` 方法的操作数应是可迭代的数值对象

In [0]:
v1 + 'ABC'

TypeError: ignored

###### 晦涩难懂的错误消息
- 示例 13-8 和示例 13-9 的错误消息晦涩难懂
- 如果由于类型不兼容而导致运算符特殊方法无法返回有效的结果,那么应该返回 `NotImplemented`,而不是抛出 `TypeError`
- 返回 `NotImplemented` 时,另一个操作数所属的类型还有机会执行运算,即 Python 会尝试调用反向方法
- 如果反向方法返回 `NotImplemented`,那么 Python 会抛出 `TypeError`,并返回一个标准的错误消息
  - 例如 “unsupported operand type(s) for +: Vector and str”。

###### 示例 13-10 vector_v6.py: `+` 运算符方法,添加到 vector_v5.py(见示例 10-16)中

In [0]:
%%writefile -a ch13/vector_v6.py


  def __add__(self, other):
    try:
      pairs = itertools.zip_longest(self, other, fillvalue=0.0)
      return Vector(a + b for a, b in pairs)
    except TypeError:
      return NotImplemented

  def __radd__(self, other):
    return self + other

Appending to ch13/vector_v6.py


In [0]:
import ch13.vector_v6
imp.reload(ch13.vector_v6)
from ch13.vector_v6 import Vector

In [0]:
a = Vector([1, 2, 3, 4])
b = Vector2d(1, 2)
b + a

Vector([2.0, 4.0, 3.0, 4.0])

In [0]:
a + '1'

TypeError: ignored

## 13.4 重载标量运算符 `*`

###### 示例 13-11 vector_v7.py:增加 `*` 运算符方法

- 展示了白鹅类型的实际应用 -- 显式的检查抽象类型

In [0]:
!cat ch13/vector_v6.py > ch13/vector_v7.py
!sed -i "1i\import numbers" ch13/vector_v7.py

In [0]:
%%writefile -a ch13/vector_v7.py


  def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
      return Vector(n * scalar for n in self)
    else:
      return NotImplemented

  def __rmul__(self, scalar):
    return self * scalar

Appending to ch13/vector_v7.py


In [0]:
from ch13.vector_v7 import Vector
v1 = Vector([1.0, 2.0, 3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [0]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [0]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

###### 表13-1:中缀运算符方法的名称(就地运算符用于增量赋值;比较运算符在表13-2中)

| 运算符           | 正向方法             | 反向方法              | 就地方法              | 说明              |
|---------------|------------------|-------------------|-------------------|-----------------|
| \+            | \_\_add\_\_      | \_\_radd\_\_      | \_\_iadd\_\_      | 加法或拼接           |
| \-            | \_\_sub\_\_      | \_\_rsub\_\_      | \_\_isub\_\_      | 减法              |
| \*            | \_\_mul\_\_      | \_\_rmul\_\_      | \_\_imul\_\_      | 乘法或重复复制         |
| /             | \_\_truediv\_\_  | \_\_rtruediv\_\_  | \_\_itruediv\_\_  | 除法              |
| //            | \_\_floordiv\_\_ | \_\_rfloordiv\_\_ | \_\_ifloordiv\_\_ | 整除              |
| %             | \_\_mod\_\_      | \_\_rmod\_\_      | \_\_imod\_\_      | 取模              |
| divmod\(\)    | \_\_divmod\_\_   | \_\_rdivmod\_\_   | \_\_idivmod\_\_   | 返回由整除的商和模数组成的元组 |
| \*\*, pow\(\) | \_\_pow\_\_      | \_\_rpow\_\_      | \_\_ipow\_\_      | 取幂              |
| @             | \_\_matmul\_\_   | \_\_rmatmul\_\_   | \_\_imatmul\_\_   | 矩阵乘法            |
| &             | \_\_and\_\_      | \_\_rand\_\_      | \_\_iand\_\_      | 位与              |
| \|            | \_\_or\_\_       | \_\_ror\_\_       | \_\_ior\_\_       | 位或              |
| ^             | \_\_xor\_\_      | \_\_rxor\_\_      | \_\_ixor\_\_      | 位异或             |
| <<            | \_\_lshift\_\_   | \_\_rlshift\_\_   | \_\_ilshift\_\_   | 按位左移            |
| >>            | \_\_rshift\_\_   | \_\_rrshift\_\_   | \_\_irshift\_\_   | 按位右移            |


###### Python 3.5 引入的中缀运算符 `@`

- `@` 运算符由特殊方法 `__matmul__`、`__rmatmul__` 和 `__imatmul__` 提供支持,名称取自“matrix multiplication”(矩阵乘法)。目前,标准库还没用到这些方法,但是 Python 3.5 的解释器能识别

In [0]:
!cat ch13/vector_v7.py > ch13/vector_py3_5.py

In [0]:
%%writefile -a ch13/vector_py3_5.py


  def __matmul__(self, other):
    try:
      return sum(a*b for a, b in zip(self, other))
    except TypeError:
      return NotImplemented

  def __rmatmul__(self, other):
    return self @ other

Appending to ch13/vector_py3_5.py


In [0]:
from ch13.vector_py3_5 import Vector

In [0]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz == 38.0  # 1 * 5 + 2 * 6 + 3 * 7

True

In [0]:
[10, 20, 30] @ vz

380.0

In [0]:
va @ 3

TypeError: ignored

## 13.5 众多比较运算符

- Python 解释器对众多比较运算符(`==`、`!=`、`>`、`<`、`>=`、`<=`)的处理与前文类似,不过在两个方面有重大区别
  1. 正向和反向调用使用的是同一系列方法。这方面的规则如表 13-2所示
    - 例如,对 `==` 来说,正向和反向调用都是 `__eq__` 方法,只是把参数对调了
    - 而正向的 `__gt__` 方法调用的是反向的 `__lt__` 方法,并把参数对调了
  2. 对 `==` 和 `!=` 来说,如果反向调用失败,Python 会比较对象的 ID,而不抛出 TypeError
- [`functools.total_ordering`](https://docs.python.org/3/library/functools.html#functools.total_ordering) 函数是个类装饰器(Python 2.7 及以上版本可用), 它能为只定义了几个比较运算符的类自动生成全部比较运算符

###### 表13-2:众多比较运算符:正向方法返回 `NotImplemented`的话,调用反向方法

| 分组  | 中缀运算符   | 正向方法调用             | 反向方法调用             | 后备机制                  |
|-----|---------|--------------------|--------------------|-----------------------|
| 相等性 | a == b  | a\.\_\_eq\_\_\(b\) | b\.\_\_eq\_\_\(a\) | 返回 id\(a\) == id\(b\) |
|     | a \!= b | a\.\_\_ne\_\_\(b\) | b\.\_\_ne\_\_\(a\) | 返回 not \(a == b\)     |
| 排序  | a > b   | a\.\_\_gt\_\_\(b\) | b\.\_\_lt\_\_\(a\) | 抛出 TypeError          |
|     | a < b   | a\.\_\_lt\_\_\(b\) | b\.\_\_gt\_\_\(a\) | 抛出 TypeError          |
|     | a >= b  | a\.\_\_ge\_\_\(b\) | b\.\_\_le\_\_\(a\) | 抛出 TypeError          |
|     | a <= b  | a\.\_\_le\_\_\(b\) | b\.\_\_ge\_\_\(a\) | 抛出 TypeError          |


- Python 2 之后的比较运算符后备机制都变了
- 对于 `__ne__`,现在 Python 3 返回结果是对 `__eq__` 结果的取反
- 对于排序比较运算符,Python 3 抛出 `TypeError`,并把错误消息设为 'unorderable types: int() < tuple()'

- Vector_v5.py 中的 `__eq__` 方法
  - ```python
    class Vector:
      # many lines omitted
      def __eq__(self, other):
        return (len(self) == len(other) and
        all(a == b for a, b in zip(self, other)))
  ```

In [0]:
from ch10.vector_v5 import Vector
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [0]:
vc = Vector([1, 2])
from ch9.vector2d_v3 import Vector2d

def vector2d_len(self):
  return 2

Vector2d.__len__ = vector2d_len  # 为了避免后续出较出错

In [0]:
v2d = Vector2d(1, 2)
vc == v2d  # 如果 Vector 实例的分量与 Vector2d 实例的分量都相等,那么两个实例相等

True

In [0]:
t3 = (1, 2, 3)
va == t3  # Vector 实例的分量与元组或其他任何可迭代对象的元素相等,那么对象也相等

True

###### 示例 13-13 vector_v8.py:改进 `Vector` 类的 `__eq__` 方法

In [0]:
!sed '/def __eq__(self, other):/d;return len(self) == len(other) and all(a == b for a, b in zip(self, other))/d' ch13/vector_v7.py > ch13/vector_v8.py

In [0]:
%%writefile -a ch13/vector_v8.py


  def __eq__(self, other):
    if isinstance(other, Vector):
      return (len(self)  == len(other)) and all(a == b for a, b in zip(self, other))
    else:
      return NotImplemented

Appending to ch13/vector_v8.py


###### 示例 13-14 与示例 13-12 一样的测试:最后一个结果变了

In [0]:
import ch13.vector_v8
imp.reload(ch13.vector_v8)
from ch13.vector_v8 import Vector
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [0]:
from ch9.vector2d_v3 import Vector2d
vc = Vector([1, 2])
v2d = Vector2d(1, 2)
vc == v2d

True

1. 为了计算 `vc == v2d`, Python 调用 `Vector.__eq__(vc, v2d)`
2. 经 `Vector.__eq__(vc, v2d)` 确认, `v2d` 不是 `Vector` 实例,因此返回 `NotImplemented`
3. Python 得到 `NotImplemented` 结果,尝试调用 `Vector2d.__eq__(v2d, vc)`
4. `Vector2d.__eq__(v2d, vc)` 把两个操作数都变成元组,然后比较,结果是 `True`

In [0]:
t3 = (1, 2, 3)
va == t3

False

1. 为了计算 `va == t3`, Python 调用 `Vector.__eq__(va, t3)` 
2. 经 `Vector.__eq__(va, t3)` 确认, `t3` 不是 `Vector` 实例,因此返回 `NotImplemented`
3. Python 得到 `NotImplemented` 结果, 尝试调用 `tuple.__eq__(t3, va)`
4. `tuple.__eq__(t3, va)` 不知道 Vector 是什么,因此返回 `NotImplemented`
5. 对 `==` 来说,如果反向调用返回 `NotImplemented`,Python 会比较对象的 ID,作最后一搏

###### `__ne__` 方法

- 一般不会实现 `!=` 运算符,因为从 `object` 继承的 `__ne__` 方法的后备行为满足了我们的需求
  - 定义了 `__eq__` 方法,而且它不返回 `NotImplemented`, `__ne__` 会对 `__eq__` 返回的结果取反

In [0]:
va != vb

False

In [0]:
vc != v2d

False

In [0]:
va != (1, 2, 3)

True

- 从 `object` 中继承的 `__ne__` 方法,运作方式与下述代码类似,不过原版是用 C 语言实现的
``` python
  def __ne__(self, other):
    eq_result = self == other
    if eq_result is NotImplemented:
      return NotImplemented
    else:
      return not eq_result
```

## 13.6 增量赋值运算符

###### 示例 13-15 增量赋值不会修改不可变目标,而是新建实例,然后重新绑定

In [4]:
from ch13.vector_v8 import Vector
v1 = Vector([1, 2, 3])
v1_alias = v1  # 复制一份
id(v1)

140659028772176

In [5]:
v1 += Vector([4, 5, 6])
v1  # 结果与预期相符

Vector([5.0, 7.0, 9.0])

In [6]:
id(v1)  # 但是创建了新的 Vector 实例

140659015916064

In [8]:
v1_alias  # 审查 v1_alias,确认原来的 Vector 实例没被修改

Vector([1.0, 2.0, 3.0])

In [9]:
id(v1_alias)

140659028772176

In [10]:
v1 *= 11
v1

Vector([55.0, 77.0, 99.0])

In [11]:
id(v1)

140659028771392

###### 增量赋值运算符的调用过程

- 如果一个类没有实现表 13-1 列出的就地运算符,增量赋值运算符只是语法糖
  - `a += b` 的作用与 `a = a + b` 完全一样
  - 对不可变类型来说,这是预期的行为
  - 如果定义了 __add__ 方法的话,不用编写额外的代码,+= 就能使用
- 如果实现了就地运算符方法,例如 `__iadd__`,计算 `a += b` 的结果时会调用就地运算符方法
  - 它们会就地修改左操作数,而不会创建新对象作为结果
- 对于不可变类型,一定不能实现就地特殊方法

###### 示例 13-18 bingoaddable.py: `AddableBingoCage` 扩展 `BingoCage`,支持 `+` 和 `+=`

In [10]:
%%writefile ch13/bingoaddable.py
from ch11.tombola import Tombola
from ch11.bingo import BingoCage


class AddableBingoCage(BingoCage):
  
  def __add__(self, other):
    if isinstance(other, Tombola):
      return AddableBingoCage(self.inspect() + other.inspect())
    else:
      return NotImplemented

  def __iadd__(self, other):
    if isinstance(other, Tombola):
      other_iterable = other.inspect()
    else:
      try:
        other_iterable = iter(other)
      except TypeError:
        self_cls = type(self).__name__
        msg = "right operand in += must be {!r} or an iterable"
        raise TypeError(msg.format(self_cls))
    self.load(other_iterable)
    return self

Overwriting ch13/bingoaddable.py


###### 示例 13-16 使用 `+` 运算符新建 `AddableBingoCage` 实例

In [0]:
import ch13.bingoaddable
imp.reload(ch13.bingoaddable)
from ch13.bingoaddable import AddableBingoCage

In [13]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [14]:
globe.pick() in vowels

True

In [15]:
len(globe.inspect())

4

In [0]:
globe2 = AddableBingoCage('XYZ')

In [24]:
globe3 = globe + globe2
len(globe3.inspect())

7

In [19]:
void = globe + [10, 20]

TypeError: ignored

###### 示例 13-17 可以使用 `+=` 运算符载入现有的 `AddableBingoCage` 实例(接续示例 13-16)

In [25]:
globe_orig = globe
len(globe.inspect())

4

In [27]:
globe += globe2
len(globe.inspect())

10

In [28]:
globe += ['M', 'N']
len(globe.inspect())

12

In [29]:
globe is globe_orig

True

In [31]:
globe += 1  # AddableBingoCage 实例不能与非可迭代对象相加,错误消息会指明原因

TypeError: ignored

###### 反向方法

- 如果中缀运算符的正向方法(如 `__mul__`)只处理与 `self` 属于同一类型的操作数,那就无需实现对应的反向方法(如` __rmul__`)
  - 因为按照定义, 反向方法是为了处理类型不同的操作数